1. Introdução
Este notebook detalha o processo de análise de dados de acidentes de trânsito em Porto Alegre, a criação de um agente inteligente para recomendação de medidas de moderação de tráfego e a construção de um modelo preditivo para estimar a ocorrência de acidentes futuros

1.1. Configuração do Ambiente e Importação de Bibliotecas

In [ ]:
# Manipulação e Análise de Dados
import pandas as pd
import numpy as np

# Visualização de Dados
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap

# Geocodificação e APIs
import openmeteo_requests
import requests_cache
from retry_requests import retry
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Modelagem Preditiva
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Utilitários
from tqdm.notebook import tqdm
import warnings

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)
tqdm.pandas()

2. Coleta, Limpeza e Enriquecimento dos Dados

2.1. Carga e Limpeza Inicial do Dataset 

In [ ]:
# Carregar o dataset
df = pd.read_csv('acidentes.csv', sep=';', low_memory=False)

# Limpeza inicial dos nomes das colunas
df.columns = df.columns.str.strip().str.lower()

# Converter colunas de data e hora
df['data'] = pd.to_datetime(df['data'], errors='coerce')
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S.%f', errors='coerce').dt.time

# Combinar data e hora em uma única coluna datetime
df['data_hora'] = df.apply(lambda r: pd.datetime.combine(r['data'], r['hora']) if pd.notnull(r['data']) and pd.notnull(r['hora']) else pd.NaT, axis=1)
df = df.dropna(subset=['data_hora'])

# Tratar coordenadas inválidas (0 ou nulas)
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df.loc[df['latitude'] == 0, 'latitude'] = np.nan
df.loc[df['longitude'] == 0, 'longitude'] = np.nan

# Criar colunas agregadas para vítimas e veículos
df['total_vitimas'] = df['feridos'] + df['fatais']
veiculo_cols = ['auto', 'taxi', 'lotacao', 'onibus_urb', 'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta', 'outro', 'patinete']
df['total_veiculos'] = df[veiculo_cols].sum(axis=1)

# Criar coluna de severidade
def define_severidade(row):
    if row['fatais'] > 0:
        return 'Fatal'
    elif row['feridos'] > 0:
        return 'Com Feridos'
    else:
        return 'Sem Vítimas'
df['severidade'] = df.apply(define_severidade, axis=1)

print(f"Dataset carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")
df.head()

2.2. Enriquecimento Geoespacial via Geocodificação
Muitos registros não possuem coordenadas de latitude e longitude. Usaremos a API Nominatim (OpenStreetMap) para geocodificar os endereços (logradouros e cruzamentos) e preencher os dados faltantes.